<a href="https://colab.research.google.com/github/mph7/projeto_covid/blob/main/projeto_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto COVID-19

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
url = 'https://github.com/mph7/projeto_covid/blob/main/covid_19_data.csv?raw=true'

In [ ]:
df = pd.read_csv(url, parse_dates=['ObservationDate', 'Last Update'])
df

In [ ]:
df.dtypes

In [ ]:
import re

def corrige_colunas(col_name):
    return re.sub(r"[/| ]", "", col_name).lower()

In [ ]:
df.columns = [corrige_colunas(col) for col in df.columns]

In [ ]:
df

# Brasil

In [ ]:
df.loc[df.countryregion == 'Brazil']

In [ ]:
brasil = df.loc[
    (df.countryregion == 'Brazil') & 
    (df.confirmed > 0.0)
]
agrupado = {'confirmed': 'sum', 'deaths': 'sum', 'recovered': 'sum', 'lastupdate': 'first', 'countryregion': 'first', 'sno': 'first'}
brasil = brasil.groupby('observationdate', as_index=False).aggregate(agrupado).reindex(columns=brasil.columns)

In [ ]:
brasil

# Gráfico da evolução de casos confirmados

In [ ]:
px.line(brasil, 'observationdate', 'confirmed', title='Casos Confirmados no Brasil')

# Novos casos por dia

In [ ]:
brasil['novoscasos'] = list(map(
    lambda x: 0 if (x==0) else brasil['confirmed'].iloc[x] - brasil['confirmed'].iloc[x-1],
    np.arange(brasil.shape[0])
))

In [ ]:
px.line(brasil, 'observationdate', 'novoscasos', title='Novos Casos Confirmados no Brasil por Dia')

# Gráfico da Evolução de Mortes


In [ ]:

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=brasil.observationdate, y=brasil.deaths, name='Mortes no Brasil', mode='lines+markers', line={'color': 'red'})
)

fig.update_layout(title='Mortes por COVID-19 no Brasil')

fig.show()

In [ ]:
brasil['novasmortes'] = list(map(
    lambda x: 0 if (x==0) else brasil['deaths'].iloc[x] - brasil['deaths'].iloc[x-1],
    np.arange(brasil.shape[0])
))

px.line(brasil, 'observationdate', 'novasmortes', title='Novas Mortes Diárias por COVID-19 no Brasil')

# Taxa de crescimento

taxa_crescimento = (presente/passado)**(1/n) -1

In [ ]:
def taxa_crescimento(data, variable, data_inicio=None, data_fim=None):
  if data_inicio == None:
    data_inicio = data.observationdate.loc[data[variable] > 0].min()
  else:
    data_inicio = pd.to_datetime(data_inicio)

  if data_fim == None:
    data_fim = data.observationdate.iloc[-1]
  else:
    data_fim = pd.to_datetime(data_fim)

  passado = data.loc[data.observationdate == data_inicio, variable].values[0]
  presente = data.loc[data.observationdate == data_fim, variable].values[0]

  n = (data_fim - data_inicio).days

  taxa = (presente/passado)**(1/n) - 1

  return 100*taxa

In [ ]:
# Taxa de crescimento médio do COVID no Brasil em todo o periodo
taxa_crescimento(brasil, 'confirmed')

In [ ]:
def taxa_crescimento_diaria(data, variable, data_inicio=None):
  if data_inicio == None:
    data_inicio = data.observationdate.loc[data[variable] > 0].min()
  else:
    data_inicio = pd.to_datetime(data_inicio)

  data_fim = data.observationdate.max()
  n = (data_fim - data_inicio).days

  taxas = list(map(
      lambda x: (data[variable].iloc[x] - data[variable].iloc[x-1]) / data[variable].iloc[x-1],
      range(1, n+1)
  ))

  return np.array(taxas) * 100

In [ ]:
tx_dia = taxa_crescimento_diaria(brasil, 'confirmed')

In [ ]:
#@title Default title text
tx_dia

In [ ]:
primeiro_dia = brasil.observationdate.loc[brasil.confirmed > 0].min()

px.line(x=pd.date_range(primeiro_dia, brasil.observationdate.max())[1:],
        y=tx_dia, title='Taxa de crescimento de casos confirmados no Brasil')

# Predições

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

In [ ]:
confirmados = brasil.confirmed
confirmados.index = brasil.observationdate
confirmados

In [ ]:
res = seasonal_decompose(confirmados)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10,8))

ax1.plot(res.observed)
ax2.plot(res.trend)
ax3.plot(res.seasonal)
ax4.plot(confirmados.index, res.resid)
ax4.axhline(0, linestyle='dashed', c='black')
plt.show

# ARIMA

In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima
modelo = auto_arima(confirmados)

In [ ]:
fig = go.Figure(go.Scatter(
    x=confirmados.index, y=confirmados, name='Observados'
))

fig.add_trace(go.Scatter(
    x=confirmados.index, y=modelo.predict_in_sample(), name='Preditos'
))

fig.add_trace(go.Scatter(
    x=pd.date_range('2021-05-29', '2021-06-29'), y=modelo.predict(31), name='Forecast'
))

fig.update_layout(title='Previsão de casos confirmados no Brasil para os próximos 30 dias')

fig.show()

# Modelo de crescimento

In [ ]:
!pip install fbprophet

In [ ]:
from fbprophet import Prophet

In [ ]:
train = confirmados.reset_index()[:-5]
test = confirmados.reset_index()[-5:]

train.rename(columns={'observationdate':'ds', 'confirmed':'y'}, inplace=True)
test.rename(columns={'observationdate':'ds', 'confirmed':'y'}, inplace=True)

profeta = Prophet(growth='logistic', changepoints=['2020-03-21', '2020-03-30', '2020-4-25', 
                                                   '2020-05-03', '2020-05-10'])
pop = 211463256
train['cap'] = pop

profeta.fit(train)

future_dates = profeta.make_future_dataframe(periods=1200)
future_dates['cap'] = pop
forecast = profeta.predict(future_dates)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat, name='Prediçao'))
#fig.add_trace(go.Scatter(x=test.index, y=test, name='Observados - Teste'))
fig.add_trace(go.Scatter(x=train.ds, y=train.y, name='Observados - Treino'))
fig.update_layout(title='Prediçoes de casos confirmados no Brasil')
fig.show()